In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1=pd.read_csv(r"C:\Users\Hp\Downloads\dataset and all\training_set_features.csv")
df2=pd.read_csv(r"C:\Users\Hp\Downloads\dataset and all\training_set_labels.csv")

In [3]:
a=[col for col in df1.columns if df1[col].dtype=="O"]

In [4]:
d={'age_group':{'65+ Years':65,'55 - 64 Years':55,'45 - 54 Years':45,'35 - 44 Years':35,'18 - 34 Years':18}}
df1=df1.replace(d)

In [5]:
from sklearn.preprocessing import LabelEncoder
def encode_missing_columns(df,col):
    le=LabelEncoder()
    u_without_nan=pd.Series([i for i in df[col].unique() if type(i) == str ])
    le.fit(u_without_nan)
    df[col]=df[col].apply(lambda x: le.transform([x])[0] if type(x) == str else x)

In [8]:
encode_missing_columns(df1,'education')

In [9]:
list_columns=a[2:]

In [10]:
for i in list_columns:
    encode_missing_columns(df1,i)

In [12]:
from sklearn.impute import SimpleImputer
def fill_with_mean(df,col):
    imp1=SimpleImputer(strategy="mean")
    df[[col]]=imp1.fit_transform(df[[col]])
    return df

def fill_with_rounded_mean(df,col):
    means=round(df[col].mean())
    df[col].fillna(means,inplace=True)
    return df

In [13]:
list_mean=['xyz_concern','xyz_knowledge','behavioral_avoidance','behavioral_wash_hands','behavioral_large_gatherings',\
           'behavioral_outside_home','behavioral_touch_face','child_under_6_months','opinion_xyz_vacc_effective',\
           'opinion_xyz_risk','opinion_xyz_sick_from_vacc','opinion_seas_vacc_effective','opinion_seas_risk',\
           'opinion_seas_sick_from_vacc','income_poverty']
list_roundedmean=['behavioral_antiviral_meds','behavioral_face_mask','doctor_recc_xyz','doctor_recc_seasonal',\
            'chronic_med_condition','health_worker','education','race','sex','marital_status','rent_or_own',\
            'employment_status','hhs_geo_region','census_msa','household_adults','household_children']

In [15]:
for i in list_mean:
    fill_with_mean(df1,i)
for j in list_roundedmean:
    fill_with_rounded_mean(df1,j)

In [19]:
from sklearn.impute import KNNImputer
k=KNNImputer(n_neighbors=5)
df1[['health_insurance','employment_industry','employment_occupation']]=k.fit_transform(df1[['health_insurance','employment_industry','employment_occupation']])

In [20]:
columns1 = [col for col in df1.columns if 'seas' in col]
columns2 = [col for col in df1.columns if 'xyz' in col]

In [22]:
from sklearn.model_selection import train_test_split
x_train1,x_test1,y_train1,y_test1=train_test_split(df1.drop(columns1,axis=1),df2['xyz_vaccine'],test_size=0.001,random_state=42)
x_train2,x_test2,y_train2,y_test2=train_test_split(df1.drop(columns2,axis=1),df2['seasonal_vaccine'],test_size=0.001,random_state=42)

In [23]:
from sklearn.linear_model import LogisticRegression
le1=LogisticRegression(max_iter=1000)
le1.fit(x_train1,y_train1)
pr1=le1.predict_proba(x_test1)
le2=LogisticRegression(max_iter=1000)
le2.fit(x_train2,y_train2)
pr2=le2.predict_proba(x_test2)

In [24]:
from sklearn.metrics import roc_auc_score
a1=roc_auc_score(y_test1,pr1[:,1])
a2=roc_auc_score(y_test2,pr2[:,1])
print(a1)
print(a2)
print((a1+a2)/2)

0.9736842105263158
0.8522727272727273
0.9129784688995215


In [25]:
df_test=pd.read_csv(r"C:\Users\Hp\Downloads\dataset and all\test_set_features.csv")

In [26]:
df_test=df_test.replace(d)

In [27]:
from sklearn.preprocessing import LabelEncoder
def encode_missing_columns(df,col):
    le=LabelEncoder()
    u_without_nan=pd.Series([i for i in df[col].unique() if type(i) == str ])
    le.fit(u_without_nan)
    df[col]=df[col].apply(lambda x: le.transform([x])[0] if type(x) == str else x)

In [29]:
encode_missing_columns(df_test,'education')
for i in list_columns:
    encode_missing_columns(df_test,i)
for i in list_mean:
    fill_with_mean(df_test,i)
for j in list_roundedmean:
    fill_with_rounded_mean(df_test,j)

In [30]:
from sklearn.impute import KNNImputer
k=KNNImputer(n_neighbors=5)
df_test[['health_insurance','employment_industry','employment_occupation']]=k.fit_transform(df_test[['health_insurance','employment_industry','employment_occupation']])

In [31]:
X_TEST1=df_test.drop(columns1,axis=1)
X_TEST2=df_test.drop(columns2,axis=1)
pred1=le1.predict_proba(X_TEST1)
pred2=le2.predict_proba(X_TEST2)

In [32]:
df3=pd.read_csv(r"C:\Users\Hp\Downloads\dataset and all\submission_format.csv")

In [34]:
df3['h1n1_vaccine']=pred1[:,1]
df3['xyz_vaccine']=pred1[:,1]
df3['seasonal_vaccine']=pred2[:,1]
df3.to_csv(r"C:\Users\Hp\Downloads\dataset and all\submission_format.csv",index=False)